Lớp **NHẬP MÔN HỌC MÁY - CQ2017/21**  
Học kì II, 2019 - 2020  

<hr>
<br>
<center><b><font size="10">ĐỒ ÁN CUỐI KÌ</font></b></center>
<br>
<center>
    <font size="4"><b>Nhóm thực hiện:</b></font><br>
    Trần Minh Trí - 1712834&emsp;Nguyễn Nhật Trường - 1712852<br>
</center>
<hr>

In [1]:
import numpy as np
import pickle
import os.path
import gzip
import math
import sklearn
from sklearn import svm
import time

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
print("Load model phai cung version 0.23.2")
sklearn.__version__

Load model phai cung version 0.23.2


'0.23.2'

In [3]:
def read_mnist(mnist_file):
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    
    train_X, train_Y = train_data
    val_X, val_Y = val_data
    test_X, test_Y = test_data    
    
    return train_X, train_Y, val_X, val_Y, test_X, test_Y

In [4]:
# Test
train_X, train_Y, val_X, val_Y, test_X, test_Y = read_mnist('mnist.pkl.gz')

print('train_X.shape =', train_X.shape)
print('train_Y.shape =', train_Y.shape)
print('val_X.shape   =', val_X.shape)
print('val_Y.shape   =', val_Y.shape)
print('test_X.shape  =', test_X.shape)
print('test_Y.shape  =', test_Y.shape)

print('\ntrain_X: min = %.3f, max = %.3f' %(train_X.min(), train_X.max()))
print('train_Y: min = %d, max = %d' %(train_Y.min(), train_Y.max()))

train_X.shape = (50000, 784)
train_Y.shape = (50000,)
val_X.shape   = (10000, 784)
val_Y.shape   = (10000,)
test_X.shape  = (10000, 784)
test_Y.shape  = (10000,)

train_X: min = 0.000, max = 0.996
train_Y: min = 0, max = 9


# Huấn luyện SVM Linear

In [5]:
# Hàm huấn luyện save từng model lại để:
#   - Nếu có lỗi giữa chừng (máy treo,...) vẫn còn 1 phần tiến trình khi chạy lại
#   - Không cần train lại lỗi lần cần gọi lại model
def create_linear_clf(c):
    to_save = "linear_data/linear_C={0}.pkl".format(c)
    if not os.path.isfile(to_save):
        starttime = time.time()
        print("Started", to_save, "at", time.strftime('%H:%M:%S', time.localtime(starttime)), end=" | ")
        model = svm.SVC(kernel="linear", decision_function_shape='ovo', C=c)
        model.fit(train_X, train_Y)
        pickle.dump(model, open(to_save, 'wb'))
        print("Finished. Time elapsed:", time.time() - starttime)
    else:
        model = pickle.load(open(to_save, 'rb'))
        print("Loaded: ", to_save)
    return model

In [6]:
clf_linear_list = []
C = [20, 10, 5, 4, 3, 2, 1, 0.5, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
for c in C:
    model =  create_linear_clf(c)
        
    clf_linear_list.append(model)

Loaded:  linear_data/linear_C=20.pkl
Loaded:  linear_data/linear_C=10.pkl
Loaded:  linear_data/linear_C=5.pkl
Loaded:  linear_data/linear_C=4.pkl
Loaded:  linear_data/linear_C=3.pkl
Loaded:  linear_data/linear_C=2.pkl
Loaded:  linear_data/linear_C=1.pkl
Loaded:  linear_data/linear_C=0.5.pkl
Loaded:  linear_data/linear_C=0.1.pkl
Loaded:  linear_data/linear_C=0.01.pkl
Loaded:  linear_data/linear_C=0.001.pkl
Loaded:  linear_data/linear_C=0.0001.pkl
Loaded:  linear_data/linear_C=1e-05.pkl
Loaded:  linear_data/linear_C=1e-06.pkl


In [7]:
scr_val = []

In [8]:
%%time # tb: 41min 15s
for clf in clf_linear_list:
    scr_val.append(clf.score(val_X, val_Y))

Wall time: 41min 15s


In [9]:
print("C", "Val", sep='\t')
for i in range(len(clf_linear_list)):
    print(C[i], scr_val[i],sep='\t')

C	Val
20	0.9264
10	0.9284
5	0.9306
4	0.9325
3	0.9347
2	0.9377
1	0.9423
0.5	0.9445
0.1	0.9481
0.01	0.9437
0.001	0.9309
0.0001	0.8971
1e-05	0.6053
1e-06	0.1064


In [12]:
max_s = max(scr_val)
max_i = scr_val.index(max_s)
max_s, max_i, clf_linear_list[max_i]

(0.9481, 8, SVC(C=0.1, decision_function_shape='ovo', kernel='linear'))

In [13]:
clf_linear_list2 = []
C = [0.12, 0.11, 0.1, 0.09, 0.08]
for c in C:
    model = create_linear_clf(c)
        
    clf_linear_list2.append(model)

Loaded:  linear_data/linear_C=0.12.pkl
Loaded:  linear_data/linear_C=0.11.pkl
Loaded:  linear_data/linear_C=0.1.pkl
Loaded:  linear_data/linear_C=0.09.pkl
Loaded:  linear_data/linear_C=0.08.pkl


In [14]:
scr_val = []

In [15]:
%%time # tb 7min 23s
for clf in clf_linear_list2:
    scr_val.append(clf.score(val_X, val_Y))

Wall time: 7min 23s


In [16]:
print("C", "Val", sep='\t')
for i in range(len(clf_linear_list2)):
    print(C[i], scr_val[i],sep='\t')

C	Val
0.12	0.9472
0.11	0.9477
0.1	0.9481
0.09	0.948
0.08	0.9478


In [17]:
clf_linear_list3 = []
C = [0.102, 0.101, 0.1, 0.099, 0.098]
for c in C:
    model = create_linear_clf(c)
        
    clf_linear_list3.append(model)

Loaded:  linear_data/linear_C=0.102.pkl
Loaded:  linear_data/linear_C=0.101.pkl
Loaded:  linear_data/linear_C=0.1.pkl
Loaded:  linear_data/linear_C=0.099.pkl
Loaded:  linear_data/linear_C=0.098.pkl


In [18]:
scr_val = []

In [19]:
%%time # tb 8min 8s
for clf in clf_linear_list3:
    scr_val.append(clf.score(val_X, val_Y))

Wall time: 8min 8s


In [20]:
print("C", "Val", sep='\t')
for i in range(len(clf_linear_list3)):
    print(C[i], scr_val[i],sep='\t')

C	Val
0.102	0.9479
0.101	0.948
0.1	0.9481
0.099	0.9482
0.098	0.9481


<b>Vậy chọn C = 0.099</b>

In [21]:
clf_linear_final = create_linear_clf(0.099)

Loaded:  linear_data/linear_C=0.099.pkl


# Huấn luyện SVM RBF

In [22]:
def create_clf(c, g):
    to_save = "rbf_data/C={0}_gamma={1}.pkl".format(c,g)
    if not os.path.isfile(to_save):
        starttime = time.time()
        print("Started", to_save, "at", time.strftime('%H:%M:%S', time.localtime(starttime)), end=" | ")
        model = svm.SVC(kernel="rbf", decision_function_shape='ovo', C=c, gamma=g)
        model.fit(train_X, train_Y)
        pickle.dump(model, open(to_save, 'wb'))
        print("Finished. Time elapsed:", time.time() - starttime)
    else:
        model = pickle.load(open(to_save, 'rb'))
        print("Loaded: ", to_save)
    return model

## Tìm tham số

<b> Lần 1 - Thử C: </b>

In [23]:
clf_rbf_list1 = []
C = [3, 2, 1, 0.5, 1e-1, 1e-2, 1e-3]
g = 0.1

for c in C:
    clf = create_clf(c, g)
    clf_rbf_list1.append(clf)

Loaded:  rbf_data/C=3_gamma=0.1.pkl
Loaded:  rbf_data/C=2_gamma=0.1.pkl
Loaded:  rbf_data/C=1_gamma=0.1.pkl
Loaded:  rbf_data/C=0.5_gamma=0.1.pkl
Loaded:  rbf_data/C=0.1_gamma=0.1.pkl
Loaded:  rbf_data/C=0.01_gamma=0.1.pkl
Loaded:  rbf_data/C=0.001_gamma=0.1.pkl


In [24]:
scr_val_l1 = []
starttime = time.time()
for clf in clf_rbf_list1:
    scr_val_l1.append(clf.score(val_X, val_Y))
print("Time:", time.time() - starttime)

Time: 2661.8074884414673


In [25]:
print("Gamma =", g)
print("C", "Val", sep='\t')
for i in range(len(clf_rbf_list1)):
    print(C[i], scr_val_l1[i],sep='\t')

Gamma = 0.1
C	Val
3	0.9566
2	0.9566
1	0.9552
0.5	0.9229
0.1	0.6875
0.01	0.2172
0.001	0.1064


-> C > 3

<b> <b> Lần 1 - Thử gamma </b>

In [26]:
clf_rbf_list2 = []
c= 0.1
G = [10, 5, 1, 1e-1, 1e-2, 1e-3]

for g in G:
    clf = create_clf(c, g)
    clf_rbf_list2.append(clf)

Loaded:  rbf_data/C=0.1_gamma=10.pkl
Loaded:  rbf_data/C=0.1_gamma=5.pkl
Loaded:  rbf_data/C=0.1_gamma=1.pkl
Loaded:  rbf_data/C=0.1_gamma=0.1.pkl
Loaded:  rbf_data/C=0.1_gamma=0.01.pkl
Loaded:  rbf_data/C=0.1_gamma=0.001.pkl


In [27]:
scr_val_l2 = [] # tb 1859s
starttime = time.time()
for clf in clf_rbf_list2:
    scr_val_l2.append(clf.score(val_X, val_Y))
print("Time:", time.time() - starttime)

Time: 1859.4848289489746


In [28]:
print("C =", c)
print("Gamma", "Val", sep='\t')
for i in range(len(clf_rbf_list2)):
    print(G[i], scr_val_l2[i],sep='\t')

C = 0.1
Gamma	Val
10	0.1064
5	0.1064
1	0.1064
0.1	0.6875
0.01	0.9578
0.001	0.9139


<b> Lần 2 - Thử C: </b>

In [29]:
g = 0.01
C = [12, 11, 10, 9, 7, 5, 3]
clf_rbf_list3 = []

for c in C:
    clf = create_clf(c, g)
    clf_rbf_list3.append(clf)

Loaded:  rbf_data/C=12_gamma=0.01.pkl
Loaded:  rbf_data/C=11_gamma=0.01.pkl
Loaded:  rbf_data/C=10_gamma=0.01.pkl
Loaded:  rbf_data/C=9_gamma=0.01.pkl
Loaded:  rbf_data/C=7_gamma=0.01.pkl
Loaded:  rbf_data/C=5_gamma=0.01.pkl
Loaded:  rbf_data/C=3_gamma=0.01.pkl


In [30]:
scr_val_l3 = [] # tb 515s
starttime = time.time()
for clf in clf_rbf_list3:
    scr_val_l3.append(clf.score(val_X, val_Y))
print("Time:", time.time() - starttime)

Time: 515.9679684638977


In [31]:
print("Gamma =", g)
print("C", "Val", sep='\t')
for i in range(len(clf_rbf_list3)):
    print(C[i], scr_val_l3[i],sep='\t')

Gamma = 0.01
C	Val
12	0.9831
11	0.9832
10	0.9835
9	0.9832
7	0.9827
5	0.9828
3	0.9818


-> C ~= 10

<b> Lần 2 - Thử C và gamma: </b>

In [12]:
gamma = [0.0106, 0.0104, 0.0102, 0.01, 0.0098, 0.0096, 0.0094]
C = [11, 10.05, 10, 9.95, 9]

clf_rbf_list4 = []
for c in C:
    for g in gamma:
        clf = create_clf(c, g)
        clf_rbf_list4.append(clf)

Started rbf_data/C=11_gamma=0.0106.pkl at 01:21:54 | Finished. Time elapsed: 221.4893777370453
Started rbf_data/C=11_gamma=0.0104.pkl at 01:25:35 | Finished. Time elapsed: 220.03825116157532
Started rbf_data/C=11_gamma=0.0102.pkl at 01:29:15 | Finished. Time elapsed: 220.1247274875641
Started rbf_data/C=11_gamma=0.01.pkl at 01:32:55 | Finished. Time elapsed: 216.97023510932922
Started rbf_data/C=11_gamma=0.0098.pkl at 01:36:32 | Finished. Time elapsed: 216.1236982345581
Started rbf_data/C=11_gamma=0.0096.pkl at 01:40:08 | Finished. Time elapsed: 214.32086944580078
Started rbf_data/C=11_gamma=0.0094.pkl at 01:43:43 | Finished. Time elapsed: 215.721266746521
Started rbf_data/C=10.05_gamma=0.0106.pkl at 01:47:18 | Finished. Time elapsed: 220.12193250656128
Started rbf_data/C=10.05_gamma=0.0104.pkl at 01:50:58 | Finished. Time elapsed: 219.92914175987244
Loaded:  rbf_data/C=10.05_gamma=0.0102.pkl
Loaded:  rbf_data/C=10.05_gamma=0.01.pkl
Loaded:  rbf_data/C=10.05_gamma=0.0098.pkl
Started rb

In [13]:
scr_l4 = [] # tb 2490s
starttime = time.time()
for clf in clf_rbf_list4:
    scr_l4.append(clf.score(val_X, val_Y))
print("Time:", time.time() - starttime)

Time: 2490.841185092926


In [23]:
print("C/gamma",0.0106,0.0104,0.0102, 0.01, 0.0098,0.0096,0.0094,sep='\t\t',end='\n\n')
i=0
while(i<len(clf_rbf_list4)):
    print(C[int(i/7)],end="\t\t")
    for j in range(7):
        print(scr_l4[i+j],end="\t\t")
    print('\n',end='')
    i+=7

C/gamma		0.0106		0.0104		0.0102		0.01		0.0098		0.0096		0.0094

11		0.9831		0.9831		0.9832		0.9832		0.9832		0.9831		0.9833		
10.05		0.9832		0.9832		0.9834		0.9834		0.9832		0.9831		0.9832		
10		0.9832		0.9832		0.9835		0.9835		0.9833		0.983		0.9832		
9.95		0.9832		0.9832		0.9835		0.9835		0.9832		0.9829		0.9831		
9		0.9835		0.9834		0.9835		0.9832		0.983		0.9829		0.983		


In [17]:
max_s = max(scr_l4)
max_i = scr_l4.index(max_s)
max_s, max_i, clf_rbf_list4[max_i]

(0.9835, 16, SVC(C=10, decision_function_shape='ovo', gamma=0.0102))

<b> Lần 3 - Thử C và gamma: </b>

In [31]:
C=[10.01, 10, 9.99, 9.98]
gamma = [0.0102, 0.01018, 0.01014, 0.0101, 0.01008, 0.01004, 0.01]

clf_rbf_list5 = []
for c in C:
    for g in gamma:
        clf = create_clf(c, g)
        clf_rbf_list5.append(clf)

Started rbf_data/C=10.01_gamma=0.0102.pkl at 11:48:45 | Finished. Time elapsed: 226.77388405799866
Started rbf_data/C=10.01_gamma=0.01018.pkl at 11:52:32 | Finished. Time elapsed: 226.73426151275635
Started rbf_data/C=10.01_gamma=0.01014.pkl at 11:56:18 | Finished. Time elapsed: 225.8777813911438
Started rbf_data/C=10.01_gamma=0.0101.pkl at 12:00:04 | Finished. Time elapsed: 225.79959297180176
Started rbf_data/C=10.01_gamma=0.01008.pkl at 12:03:50 | Finished. Time elapsed: 226.04461359977722
Started rbf_data/C=10.01_gamma=0.01004.pkl at 12:07:36 | Finished. Time elapsed: 231.44677543640137
Started rbf_data/C=10.01_gamma=0.01.pkl at 12:11:28 | Finished. Time elapsed: 224.20754432678223
Loaded:  rbf_data/C=10_gamma=0.0102.pkl
Loaded:  rbf_data/C=10_gamma=0.01018.pkl
Loaded:  rbf_data/C=10_gamma=0.01014.pkl
Loaded:  rbf_data/C=10_gamma=0.0101.pkl
Loaded:  rbf_data/C=10_gamma=0.01008.pkl
Loaded:  rbf_data/C=10_gamma=0.01004.pkl
Loaded:  rbf_data/C=10_gamma=0.01.pkl
Loaded:  rbf_data/C=9.99

In [32]:
scr_l5 = [] # tb 2072
starttime = time.time()
for clf in clf_rbf_list5:
    scr_l5.append(clf.score(val_X, val_Y))
print("Time:", time.time() - starttime)

Time: 2072.3849124908447


In [33]:
print("C/gamma",0.0102, 0.01018, 0.01014, 0.0101, 0.01008, 0.01004, 0.01,sep='\t\t',end='\n\n')
i=0
while(i<len(clf_rbf_list5)):
    print(C[int(i/7)],end="\t\t")
    for j in range(7):
        print(scr_l5[i+j],end="\t\t")
    print('\n',end='')
    i+=7

C/gamma		0.0102		0.01018		0.01014		0.0101		0.01008		0.01004		0.01

10.01		0.9834		0.9834		0.9833		0.9834		0.9835		0.9835		0.9835		
10		0.9835		0.9834		0.9834		0.9833		0.9835		0.9835		0.9835		
9.99		0.9835		0.9834		0.9834		0.9835		0.9835		0.9835		0.9835		
9.98		0.9834		0.9835		0.9834		0.9835		0.9835		0.9835		0.9835		


In [34]:
max_s = max(scr_l5)
max_i = scr_l5.index(max_s)
max_s, max_i, clf_rbf_list5[max_i]

(0.9835, 4, SVC(C=10.01, decision_function_shape='ovo', gamma=0.01008))

<b>Chọn C=10, gamma = 0.01</b>

In [8]:
clf_rbf_final = create_clf(10, 0.01)

Loaded:  rbf_data/C=10_gamma=0.01.pkl


# Chọn hàm dự đoán cuối cùng

Model RBF có score trên tập validation > Model Linear, nên chọn model RBF

[Link download file lưu model này](https://drive.google.com/file/d/1T7rbMcE8leabjCkv-B139DNCnCCnq8m-/view?usp=sharing)

Độ chính xác model đã chọn trên tập test:

In [9]:
score = clf_rbf_final.score(test_X, test_Y)
score

0.982

In [10]:
print("Test error rate (%): ", (1-score)*100)

Test error rate (%):  1.8000000000000016


# Đánh giá

Mô hình được chọn có độ lỗi trên tập test khoảng 1.8%
-	Kém hơn so với mô hình đã thực hiện trước của người khác (1.4%)
-	Lý do có thể vì chưa tối ưu tốt nhất siêu tham số:
 + Ở C có đoạn C = 5 cho kết quả cao hơn so với xung quanh nhưng không có xét lân cận
 + Chia đoạn ở C = 10, γ = 0.01 chưa đủ nhỏ để có thể tìm thấy tham số tối ưu hơn.
